In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import gensim.corpora as corpora
from pprint import pprint
from gensim import  models


import pickle
from gensim.test.utils import datapath

# Import Dataset
papers = pd.read_csv('csv/data.csv')
#print(papers.head())
print(papers.head())

   Unnamed: 0                                               text
0           0  32 THIS ARTICLE HAS BEEN PEER-REVIEWED.  COMPU...
1           1  Interactions of Mean Climate Change and \nClim...
2           2  This journal is cThe Royal Society of Chemistr...
3           3  LETTERS\nPUBLISHED ONLINE: 9 MARCH 2014 | DOI:...
4           4  Del G~nio A. D. (2013) Physical processes cont...


In [2]:
# Remove punctuation
papers['text'] = \
papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['text'] = \
papers['text'].map(lambda x: x.lower())
# Print out the first rows of papers
print(papers['text'].head())


stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
             
data = papers.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
/var/folders/cv/d1w1tlhn18d523dngzfhkzcc0000gn/T/ipykernel_3438/1712462136.py:3: DeprecationWarning: invalid escape sequence \.
  papers['text'].map(lambda x: re.sub('[,\.!?]', '', x))


0    32 this article has been peer-reviewed  comput...
1    interactions of mean climate change and \nclim...
2    this journal is cthe royal society of chemistr...
3    letters\npublished online: 9 march 2014 | doi:...
4    del g~nio a d (2013) physical processes contro...
Name: text, dtype: object
['article', 'peer', 'reviewed', 'computing', 'science', 'learning', 'climate', 'informatics', 'accelerating', 'discovering', 'climate', 'science', 'machine', 'learning', 'goal', 'climate', 'informatics', 'emerging', 'discipline', 'inspire', 'collaboration', 'climate', 'scientists', 'data', 'scientists', 'order', 'develop', 'tools', 'analyze', 'complex']


In [3]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])


[(0, 3), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1)]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

pprint(lda_model.print_topics())

In [5]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/wenxiuye/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/wenxiuye/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wenxiuye/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wenxiuye/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wenxiuye/opt/anaconda3

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.158261  0.137887       1        1  36.670323
18    -0.191194 -0.081850       2        1  19.665271
0     -0.000711  0.271470       3        1  12.923523
2     -0.298036 -0.034629       4        1   9.413021
8     -0.221891  0.129553       5        1   4.428379
3     -0.092097  0.173412       6        1   3.925194
13     0.119540  0.188667       7        1   3.494831
4     -0.209119  0.008285       8        1   3.278480
15     0.264999  0.150373       9        1   3.168302
14     0.039722  0.120862      10        1   1.668480
9     -0.099665 -0.170759      11        1   1.361172
1      0.093625 -0.098534      12        1   0.000345
5      0.092519 -0.098708      13        1   0.000339
19     0.088641 -0.094524      14        1   0.000339
7      0.094253 -0.098712      15        1   0.000338
10     0.094779 -0.100042      16        1   0.000337
12     0.095003 -0.099969      17        1   0.000332
16     0.095988 -0.101334      18        1   0.000332
11     0.093667 -0.098664      19        1   0.000331
17     0.098240 -0.102784      20        1   0.000331, topic_info=          Term         Freq        Total Category  logprob  loglift
181    climate  2558.000000  2558.000000  Default  30.0000  30.0000
271       data   752.000000   752.000000  Default  29.0000  29.0000
1643   forcing   508.000000   508.000000  Default  28.0000  28.0000
382         et  1496.000000  1496.000000  Default  27.0000  27.0000
33          al  1488.000000  1488.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
3874     ozone     0.000070   495.022838  Topic20  -8.8042  -3.1576
2322     atmos     0.000061   173.841581  Topic20  -8.9443  -2.2512
1397       air     0.000063   231.457983  Topic20  -8.9081  -2.5013
2206   aerosol     0.000067   462.262238  Topic20  -8.8383  -3.1233
1039  response     0.000066   360.833213  Topic20  -8.8631  -2.9004

[1585 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
7681       1  0.055224              aa
7681       2  0.069031              aa
7681       3  0.027612              aa
7681       4  0.842173              aa
0          7  0.962163            aaai
...      ...       ...             ...
9140       8  1.005752  δprecipitation
2159      10  0.516529      δtresponse
11516      1  0.037480           ﬁgure
11516      2  0.724613           ﬁgure
11516      6  0.224880           ﬁgure

[2196 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 19, 1, 3, 9, 4, 14, 5, 16, 15, 10, 2, 6, 20, 8, 11, 13, 17, 12, 18])